In [12]:
import pandas as pd

# 读取文件
file_1 = '附件1.xlsx'  # 修改为你本地的文件路径
file_2 = '附件2.xlsx'  # 修改为你本地的文件路径

# 加载Excel文件中的工作表
file_1_data = pd.ExcelFile(file_1)
file_2_data = pd.ExcelFile(file_2)

# 读取2023年农作物种植情况和2023年统计的相关数据
crop_data_2023 = file_2_data.parse('2023年的农作物种植情况')
crop_statistics_2023 = file_2_data.parse('2023年统计的相关数据')

# 计算平均销售价格，处理价格区间以及已经为数字的情况
crop_statistics_2023['平均销售价格'] = crop_statistics_2023['销售单价/(元/斤)'].apply(
    lambda x: (float(x.split('-')[0]) + float(x.split('-')[1])) / 2 if isinstance(x, str) and '-' in x else x)

# 提取关键字段用于分析
crop_summary = crop_statistics_2023[['作物编号', '作物名称', '亩产量/斤', '种植成本/(元/亩)', '平均销售价格']]

# 将2023年种植数据与作物统计数据进行合并
merged_data = pd.merge(crop_data_2023, crop_summary, on='作物编号', how='left')

# 计算每块地的总产量和预期收益
merged_data['总产量/斤'] = merged_data['种植面积/亩'] * merged_data['亩产量/斤']
merged_data['预期收益/元'] = merged_data['总产量/斤'] * merged_data['平均销售价格'] - merged_data['种植面积/亩'] * merged_data['种植成本/(元/亩)']

# 打印合并后的数据
print(merged_data.head())  # 你可以选择将数据保存为文件或进行进一步处理


  种植地块 作物编号 作物名称_x 作物类型  种植面积/亩 种植季次 作物名称_y   亩产量/斤  种植成本/(元/亩)  平均销售价格  \
0   A1    6     小麦   粮食    80.0   单季     小麦   800.0       450.0     3.5   
1   A1    6     小麦   粮食    80.0   单季     小麦   760.0       450.0     3.5   
2   A1    6     小麦   粮食    80.0   单季     小麦   720.0       450.0     3.5   
3   A2    7     玉米   粮食    55.0   单季     玉米  1000.0       500.0     3.0   
4   A2    7     玉米   粮食    55.0   单季     玉米   950.0       500.0     3.0   

     总产量/斤    预期收益/元  
0  64000.0  188000.0  
1  60800.0  176800.0  
2  57600.0  165600.0  
3  55000.0  137500.0  
4  52250.0  129250.0  


In [13]:
import pulp

# 初始化问题（最大化收益）
model = pulp.LpProblem("Maximize_Crop_Profit", pulp.LpMaximize)

# 从 merged_data 生成 land_area_dict，确保每个地块都有正确的面积
land_area_dict = merged_data.set_index('种植地块')['种植面积/亩'].to_dict()

# 获取实际数据
plot_codes = merged_data['种植地块'].unique()  # 地块编码
crop_names = merged_data['作物名称_x'].unique()  # 作物名称
crop_yield = merged_data.groupby('作物名称_x')['亩产量/斤'].mean().tolist()  # 每种作物的亩产量
crop_cost = merged_data.groupby('作物名称_x')['种植成本/(元/亩)'].mean().tolist()  # 每种作物的种植成本
crop_price = merged_data.groupby('作物名称_x')['平均销售价格'].mean().tolist()  # 每种作物的销售价格
expected_sales = merged_data.groupby('作物名称_x')['总产量/斤'].mean().tolist()  # 每种作物的预期销售量

# 使用前向填充方法来填充空缺值，确保数据完整性
merged_data.fillna(method='ffill', inplace=True)

# 清理地块编码，确保只有字符串类型的地块编码被处理
plot_codes_clean = [str(plot).strip() if isinstance(plot, str) else str(plot) for plot in plot_codes]

# 筛选出豆类作物
bean_crops = merged_data[merged_data['作物类型'].isin(['粮食（豆类）', '蔬菜（豆类）'])]['作物名称_x'].unique()

# 初始化每个地块的计数器，记录每块地连续多少年没有种植豆类
bean_counter = {plot: 0 for plot in plot_codes_clean}

# 初始化字典来记录上一年每块地种植的作物
previous_year_crops = {}

# 筛选出地块类型
abc_plots = [plot for plot in plot_codes_clean if plot[0] in ['A', 'B', 'C']]
water_irrigated_plots = [plot for plot in plot_codes_clean if plot[0] == 'D']
restricted_crops = ['大白菜', '白萝卜', '红萝卜']
normal_greenhouse_plots = [plot for plot in plot_codes_clean if plot[0] == 'E']
# 筛选出智慧大棚的地块（假设智慧大棚的标识为F）
smart_greenhouse_plots = [plot for plot in plot_codes_clean if plot[0] == 'F']
# 筛选出作物类型为食用菌的作物
mushroom_crops = merged_data[merged_data['作物类型'] == '食用菌']['作物名称_x'].unique()
# 筛选出作物类型为蔬菜的作物
vegetable_data = merged_data[merged_data['作物类型'] == '蔬菜']

grain_crops = [crop for crop in crop_names if '水稻' not in crop and '粮食' in merged_data[merged_data['作物名称_x'] == crop]['作物类型'].values]

crop_binary_vars = pulp.LpVariable.dicts("CropBinaryVars",
                                         ((plot, crop) for plot in plot_codes_clean for crop in crop_names),
                                         cat='Binary')


# 定义两个季度
seasons = [1, 2]
results = []

# 初始化豆类种植的计数器，记录每个地块在过去的年份是否种植过豆类
bean_counter = {plot: 0 for plot in plot_codes_clean}


# 循环优化每年的种植方案（2023-2030）
for year in range(2024, 2031):
    print(f"Optimizing for the year: {year}")
    # 定义一个二进制变量，表示某个地块是否完全种植豆类作物
    bean_planted_full = pulp.LpVariable.dicts(f"bean_planted_full_{year}",
                                            (plot for plot in plot_codes_clean),
                                            cat='Binary')

     # 定义种植决策变量 x_{plot, crop, season}: 每块地 i 在季节 season 种植作物 j 的面积
    crop_vars = pulp.LpVariable.dicts(f"x_{year}",
                                      ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                      lowBound=0, cat='Continuous')

    # 定义实际销售量 y_{plot_code, crop, season} 用于处理滞销问题
    sales_vars = pulp.LpVariable.dicts(f"y_{year}",
                                       ((plot, crop, season) for plot in plot_codes_clean for crop in crop_names for season in seasons),
                                       lowBound=0, cat='Continuous')

    # 目标函数：最大化每季的收益
    model += pulp.lpSum([sales_vars[plot, crop, season] * crop_price[j] - crop_vars[plot, crop, season] * crop_cost[j] 
                         for plot in plot_codes_clean for crop, j in zip(crop_names, range(len(crop_names))) for season in seasons])

    # 添加约束：实际销售量不能超过预期销售量和种植产量
    for plot in plot_codes_clean:
        for crop, j in zip(crop_names, range(len(crop_names))):
            for season in seasons:
                model += sales_vars[plot, crop, season] <= expected_sales[j], f"Sales_Limit_{plot}_{crop}_{season}_{year}"
                model += sales_vars[plot, crop, season] <= crop_vars[plot, crop, season] * crop_yield[j], f"Yield_Limit_{plot}_{crop}_{season}_{year}"

    # 每块地每季度的种植面积不能超过其总面积
    for plot in plot_codes_clean:
        for season in seasons:
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in crop_names]) <= land_area_dict.get(plot, 0), f"Land_Limit_{plot}_{season}_{year}"

    # 为每个地块添加种植面积总量的约束
    for plot in plot_codes_clean:
        # 计算地块的总种植面积：包括第一季和第二季的所有作物
        total_area_planted = pulp.lpSum([crop_vars[plot, crop, season] for crop in crop_names for season in seasons])
        # 确保总种植面积不能超过该地块的总面积
        model += total_area_planted <= land_area_dict.get(plot, 0), f"Total_Area_Limit_{plot}_{year}"
# 定义约束，确保豆类种植占满整块地
    for plot in plot_codes_clean:
        total_area = land_area_dict.get(plot, 0)  # 获取地块的总面积

        for season in seasons:
            # 约束：如果种植豆类作物，所有面积都必须种植豆类
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in bean_crops]) >= total_area * bean_planted_full[plot], f"Full_Bean_Plot_{plot}_{season}_{year}"

            # 其他作物不能与豆类同时种植
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in crop_names if crop not in bean_crops]) <= (1 - bean_planted_full[plot]) * total_area, f"No_Other_Crops_When_Bean_{plot}_{season}_{year}"

        # 确保当豆类作物没有被种植时，其他作物可以正常种植
        model += bean_planted_full[plot] <= 1, f"Bean_Binary_Limit_{plot}_{year}"

    # 检查三年内是否种植过豆类
    for plot in plot_codes_clean:
        if bean_counter[plot] >= 2:  # 如果三年内没有种植过豆类，强制种植豆类
            print(f"Plot {plot} must plant beans in year {year}.")
            model += pulp.lpSum([crop_vars[plot, crop, season] for crop in bean_crops for season in seasons]) == land_area_dict.get(plot, 0), f"Force_Bean_Planting_{plot}_{year}"
            bean_counter[plot] = 0  # 种植后重置计数器
        else:
            # 如果模型已求解，可以检查种植结果是否为豆类
            if model.status == pulp.LpStatusOptimal:
                planted_beans = any(crop_vars[plot, crop, 1].varValue is not None and crop_vars[plot, crop, 1].varValue > 0
                                    for crop in bean_crops) or any(crop_vars[plot, crop, 2].varValue is not None and crop_vars[plot, crop, 2].varValue > 0
                                    for crop in bean_crops)

                if planted_beans:
                    bean_counter[plot] = 0  # 种植了豆类，计数器归零
                else:
                    bean_counter[plot] += 1  # 没种植豆类，计数器加1

    # 添加约束：每块地最多种植 3 种作物
    max_crops_per_plot = 3
    for plot in plot_codes_clean:
        # 限制每块地最多种植 max_crops_per_plot 种作物
        model += pulp.lpSum([crop_binary_vars[plot, crop] for crop in crop_names]) <= max_crops_per_plot, f"MaxCrops_{plot}_{year}"

    # 为平旱地、梯田和山坡地的地块添加约束
    for plot in abc_plots:
        for crop in crop_names:
            # 限制只能种植粮食类作物（不包括水稻）
            if crop in grain_crops:
                # 只允许第一季种植作物，第二季种植面积必须为 0
                model += crop_vars[plot, crop, 2] == 0, f"ABC_Plot_Season2_Zero_{plot}_{crop}_{year}"
            else:
                # 不能种植非粮食作物
                model += crop_vars[plot, crop, 1] == 0, f"ABC_Plot_Grain_Only_{plot}_{crop}_{year}"

    # 在ABC类型地块的season2中不种植作物
    for plot in abc_plots:
        for crop in crop_names:
            model += crop_vars[plot, crop, 2] == 0, f"Season2_Zero_Limit_{plot}_{crop}_{year}"

    # 添加连续两年不能种植同一种作物的约束，按季度进行
    for plot in plot_codes_clean:
        if plot in previous_year_crops:
            for crop in crop_names:
                for season in seasons:
                    # 如果上一年相同季度种植了某种作物，今年同一季度不能种同一种作物
                    model += crop_vars[plot, crop, season] <= (1 - previous_year_crops[plot][crop, season]) * land_area_dict.get(plot, 0), f"Rotation_Limit_{plot}_{crop}_{season}_{year}"

    # 添加新的约束：水浇地每年可以单季种植水稻或两季种植蔬菜作物
    for plot in water_irrigated_plots:
        for crop in crop_names:
            for season in seasons:
                if '水稻' in crop:
                    # 如果种植水稻，season2 全部设置为 0
                    model += crop_vars[plot, crop, 2] == 0, f"Water_Irrigated_Season2_Zero_{plot}_{crop}_Season_{season}_Year_{year}"
                elif '蔬菜' in crop:
                    # 第一季可以种植任意蔬菜，除大白菜、白萝卜和红萝卜外
                    if season == 1 and crop not in restricted_crops:
                        model += crop_vars[plot, crop, season] >= 0, f"Water_Irrigated_Season1_Veg_{plot}_{crop}_Season_{season}_Year_{year}"
                    # 第二季只能种植大白菜、白萝卜和红萝卜
                    if season == 2 and crop in restricted_crops:
                        model += crop_vars[plot, crop, season] >= 0, f"Water_Irrigated_Season2_RestrictedVeg_{plot}_{crop}_Season_{season}_Year_{year}"

    # 确保第二季只能种植大白菜、白萝卜和红萝卜中的一种
    for plot in water_irrigated_plots:
        # 确保只能种植一种受限制的作物
        model += pulp.lpSum([crop_vars[plot, crop, 2] for crop in restricted_crops]) <= land_area_dict.get(plot, 0), f"Water_Irrigated_Season2_OneCrop_{plot}_Year_{year}"

    # 添加普通大棚的种植约束
    for plot in normal_greenhouse_plots:
        for crop in crop_names:
            for season in seasons:
                if season == 1 and crop not in restricted_crops:
                    # 第一季可以种植任何蔬菜，除大白菜、白萝卜和红萝卜外
                    model += crop_vars[plot, crop, season] >= 0, f"NormalGreenhouse_Season1_Veg_{plot}_{crop}_Season_{season}_Year_{year}"
                elif season == 2 and crop in mushroom_crops:
                    # 第二季只能种植食用菌
                    model += crop_vars[plot, crop, season] >= 0, f"NormalGreenhouse_Season2_Mushroom_{plot}_{crop}_Season_{season}_Year_{year}"
    # 确保第二季只能种植食用菌
    for plot in normal_greenhouse_plots:
        # 确保只能种植食用菌
        model += pulp.lpSum([crop_vars[plot, crop, 2] for crop in mushroom_crops]) <= land_area_dict.get(plot, 0), f"NormalGreenhouse_Season2_OnlyMushroom_{plot}_Year_{year}"
        # 筛选出不包含大白菜、白萝卜和红萝卜的蔬菜作物
    allowed_vegetable_crops = vegetable_data[~vegetable_data['作物名称_x'].isin(restricted_crops)]['作物名称_x'].unique()

    # 应用智慧大棚的种植约束：每年可以种植两季蔬菜，除大白菜、白萝卜和红萝卜外
    for plot in smart_greenhouse_plots:
        for crop in allowed_vegetable_crops:
            for season in seasons:
                # 第一季和第二季都可以种植任意蔬菜，除大白菜、白萝卜和红萝卜外
                model += crop_vars[plot, crop, season] >= 0, f"SmartGreenhouse_Season{season}_Veg_{plot}_{crop}_Year_{year}"


    # 求解每一年的优化问题
    model.solve()

    # 在求解之后保存当前年份的种植作物信息，用于下一个年份的轮作限制
    for plot in plot_codes_clean:
        previous_year_crops[plot] = {(crop, season): (crop_vars[plot, crop, season].varValue > 0) for crop in crop_names for season in seasons}

    # 输出每年优化结果
    # print(f"Year {year} results:")
    for plot in plot_codes_clean:
        for crop in crop_names:
            for season in seasons:
                print(f"Plot {plot}, Crop {crop}, Season {season}, Area: {crop_vars[plot, crop, season].varValue}")
    for plot in plot_codes_clean:  # 遍历地块
        for crop in crop_names:  # 遍历作物
            for season in seasons:  # 遍历季次（1, 2）
                # 获取每块地每季的种植面积
                area = crop_vars[plot, crop, season].varValue
                # 如果有种植面积大于0的作物，存储结果
                if area is not None and area > 0:
                    results.append([year, plot, season, crop, area])
# 将结果列表转换为 DataFrame
df_results = pd.DataFrame(results, columns=['年份', '地块', '季次', '作物名称', '种植面积'])

# 查看优化结果 DataFrame
print(df_results)
# 输出总收益
print(f"Maximized Profit for {2023}-{2030}: {pulp.value(model.objective)}")


Optimizing for the year: 2024


C:\Users\18798\AppData\Local\Temp\ipykernel_27456\4198164304.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)
C:\Users\18798\AppData\Local\Temp\ipykernel_27456\4198164304.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(method='ffill', inplace=True)


Plot A1, Crop 小麦, Season 1, Area: 4.4
Plot A1, Crop 小麦, Season 2, Area: 0.0
Plot A1, Crop 玉米, Season 1, Area: 0.45
Plot A1, Crop 玉米, Season 2, Area: 0.0
Plot A1, Crop 黄豆, Season 1, Area: 0.0
Plot A1, Crop 黄豆, Season 2, Area: 0.0
Plot A1, Crop 绿豆, Season 1, Area: 0.0
Plot A1, Crop 绿豆, Season 2, Area: 0.0
Plot A1, Crop 谷子, Season 1, Area: 10.0
Plot A1, Crop 谷子, Season 2, Area: 0.0
Plot A1, Crop 黑豆, Season 1, Area: 0.0
Plot A1, Crop 黑豆, Season 2, Area: 0.0
Plot A1, Crop 红豆, Season 1, Area: 0.0
Plot A1, Crop 红豆, Season 2, Area: 0.0
Plot A1, Crop 爬豆, Season 1, Area: 0.0
Plot A1, Crop 爬豆, Season 2, Area: 0.0
Plot A1, Crop 高粱, Season 1, Area: 0.6
Plot A1, Crop 高粱, Season 2, Area: 0.0
Plot A1, Crop 黍子, Season 1, Area: 21.0
Plot A1, Crop 黍子, Season 2, Area: 0.0
Plot A1, Crop 莜麦, Season 1, Area: 0.45
Plot A1, Crop 莜麦, Season 2, Area: 0.0
Plot A1, Crop 大麦, Season 1, Area: 0.0
Plot A1, Crop 大麦, Season 2, Area: 0.0
Plot A1, Crop 荞麦, Season 1, Area: 42.5
Plot A1, Crop 荞麦, Season 2, Area: 0.0
Plot A1

c:\Users\18798\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Plot A1, Crop 小麦, Season 1, Area: 0.0
Plot A1, Crop 小麦, Season 2, Area: 0.0
Plot A1, Crop 玉米, Season 1, Area: 0.0
Plot A1, Crop 玉米, Season 2, Area: 0.0
Plot A1, Crop 黄豆, Season 1, Area: 0.0
Plot A1, Crop 黄豆, Season 2, Area: 0.0
Plot A1, Crop 绿豆, Season 1, Area: 0.0
Plot A1, Crop 绿豆, Season 2, Area: 0.0
Plot A1, Crop 谷子, Season 1, Area: 0.0
Plot A1, Crop 谷子, Season 2, Area: 0.0
Plot A1, Crop 黑豆, Season 1, Area: 0.0
Plot A1, Crop 黑豆, Season 2, Area: 0.0
Plot A1, Crop 红豆, Season 1, Area: 0.0
Plot A1, Crop 红豆, Season 2, Area: 0.0
Plot A1, Crop 爬豆, Season 1, Area: 0.0
Plot A1, Crop 爬豆, Season 2, Area: 0.0
Plot A1, Crop 高粱, Season 1, Area: 0.0
Plot A1, Crop 高粱, Season 2, Area: 0.0
Plot A1, Crop 黍子, Season 1, Area: 0.0
Plot A1, Crop 黍子, Season 2, Area: 0.0
Plot A1, Crop 莜麦, Season 1, Area: 0.0
Plot A1, Crop 莜麦, Season 2, Area: 0.0
Plot A1, Crop 大麦, Season 1, Area: 25.0
Plot A1, Crop 大麦, Season 2, Area: 0.0
Plot A1, Crop 荞麦, Season 1, Area: 0.0
Plot A1, Crop 荞麦, Season 2, Area: 0.0
Plot A1, Cr

In [14]:

import pandas as pd
from openpyxl import load_workbook

df_results = pd.DataFrame(results, columns=['年份', '地块', '季次', '作物名称', '种植面积'])
def filter_rows(row):
    plot = row['地块']
    area = row['种植面积']
    land_area = land_area_dict.get(plot, 0)  # 获取地块的面积，默认为0以防止 KeyError
    # 忽略种植面积小于地块面积 15% 的行
    return area >= 0.1 * land_area

# 应用过滤条件
df_filtered_results = df_results[df_results.apply(filter_rows, axis=1)]

# 查看过滤后的结果
for index,row in df_filtered_results.iterrows():
    print(row)

年份      2024
地块        A1
季次         1
作物名称      谷子
种植面积    10.0
Name: 2, dtype: object
年份      2024
地块        A1
季次         1
作物名称      黍子
种植面积    21.0
Name: 4, dtype: object
年份      2024
地块        A1
季次         1
作物名称      荞麦
种植面积    42.5
Name: 6, dtype: object
年份      2024
地块        A2
季次         1
作物名称      谷子
种植面积    10.0
Name: 11, dtype: object
年份      2024
地块        A2
季次         1
作物名称      黍子
种植面积    21.0
Name: 13, dtype: object
年份      2024
地块        A2
季次         1
作物名称      荞麦
种植面积    17.5
Name: 15, dtype: object
年份      2024
地块        A3
季次         1
作物名称      小麦
种植面积     4.4
Name: 18, dtype: object
年份      2024
地块        A3
季次         1
作物名称      谷子
种植面积    10.0
Name: 20, dtype: object
年份      2024
地块        A3
季次         1
作物名称      黍子
种植面积    18.5
Name: 22, dtype: object
年份      2024
地块        A4
季次         1
作物名称      谷子
种植面积    10.0
Name: 28, dtype: object
年份      2024
地块        A4
季次         1
作物名称      黍子
种植面积    21.0
Name: 30, dtype: object
年份      2024
地块        A

In [15]:
from openpyxl import load_workbook

file_path = 'result1_1.xlsx'

book = load_workbook(file_path)

crop_names = df_filtered_results['作物名称'].unique().tolist()

plot_to_row = {
    'A1': 2, 'A2': 3, 'A3': 4, 'A4': 5, 'A5': 6, 'A6': 7,
    'B1': 8, 'B2': 9, 'B3': 10, 'B4': 11, 'B5': 12, 'B6': 13,
    'B7': 14, 'B8': 15, 'B9': 16, 'B10': 17, 'B11': 18, 'B12': 19,
    'B13': 20, 'B14': 21,
    'C1': 22, 'C2': 23, 'C3': 24, 'C4': 25, 'C5': 26, 'C6': 27,
    'D1': 28, 'D2': 29, 'D3': 30, 'D4': 31, 'D5': 32, 'D6': 33,
    'D7': 34, 'D8': 35,
    'E1': 36, 'E2': 37, 'E3': 38, 'E4': 39, 'E5': 40, 'E6': 41,
    'E7': 42, 'E8': 43, 'E9': 44, 'E10': 45, 'E11': 46, 'E12': 47,
    'E13': 48, 'E14': 49, 'E15': 50, 'E16': 51,
    'F1': 52, 'F2': 53, 'F3': 54, 'F4': 55
}

# 地块编号 -> Excel 行号的映射表 (季次2，从56行开始)
plot_to_row_season2 = {
    'D1': 56, 'D2': 57, 'D3': 58, 'D4': 59, 'D5': 60, 'D6': 61, 'D7': 62, 'D8': 63,
    'E1': 64, 'E2': 65, 'E3': 66, 'E4': 67, 'E5': 68, 'E6': 69, 'E7': 70, 'E8': 71,
    'E9': 72, 'E10': 73, 'E11': 74, 'E12': 75, 'E13': 76, 'E14': 77, 'E15': 78, 'E16': 79,
    'F1': 80, 'F2': 81, 'F3': 82, 'F4': 83
}

# 作物名称 -> Excel 列号的映射表
crop_to_col = {
    '黄豆': 3, '黑豆': 4, '红豆': 5, '绿豆': 6, '爬豆': 7, '小麦': 8, '玉米': 9, '谷子': 10, '高粱': 11,
    '黍子': 12, '荞麦': 13, '南瓜': 14, '红薯': 15, '莜麦': 16, '大麦': 17, '水稻': 18, '豇豆': 19, '刀豆': 20,
    '芸豆': 21, '土豆': 22, '西红柿': 23, '茄子': 24, '菠菜': 25, '青椒': 26, '菜花': 27, '包菜': 28, '油麦菜': 29,
    '小青菜': 30, '黄瓜': 31, '生菜': 32, '辣椒': 33, '空心菜': 34, '黄心菜': 35, '芹菜': 36, '大白菜': 37, '白萝卜': 38,
    '红萝卜': 39, '榆黄菇': 40, '香菇': 41, '白灵菇': 42, '羊肚菌': 43
}


def get_y_coord(plot, season):
    if season == 1:
        return plot_to_row.get(plot, 2)  # 获取季次1对应的行号
    elif season == 2:
        return plot_to_row_season2.get(plot, 56)  # 获取季次2对应的行号，默认从第56行开始
# 填充表格为0的函数
def fill_sheet_with_zeros(sheet):
    for row in range(2, 84):  # 遍历行（C2 到 AP83）
        for col in range(3, 44):  # 遍历列（C 到 AP）
            sheet.cell(row=row, column=col).value = 0  # 填入0
# 遍历所有年份的表格，先填充为0
for year in range(2024, 2031):  # 假设年份从2024到2030
    sheet_name = str(year)
    sheet = book[sheet_name]
    fill_sheet_with_zeros(sheet)  # 将C2到AP83单元格填充为0
for _, row in df_filtered_results.iterrows():
    year = row['年份']
    plot = row['地块']
    season = row['季次']
    crop = row['作物名称']
    area = row['种植面积']
    sheet_name = str(year)
    sheet = book[sheet_name]
    # 根据地块和季次确定行数（Y 坐标）
    if season == 2 and plot[0] in ['A','B','C']:
        continue
    y_coord = get_y_coord(plot, season)
        # 根据作物名称确定列数（X 坐标）
    x_coord = crop_to_col.get(crop)  # 通过作物名称获取列号
        # 将种植面积写入对应的单元格
    sheet.cell(row=y_coord, column=x_coord).value = area
        # 保存修改后的 Excel 文件
book.save(file_path)
book.close()

print("结果已成功写入 Excel 文件！")